# Random Forest Classification

## Importing the libraries

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

In [2]:
%cd Z:\PPMI_Data\Excels\Only_Male\GeneCpG\SymbDigi
df = pd.read_csv('SymbDigiCateg_Methylome_Male.csv')
print(df.shape)
df.head(3)

Z:\PPMI_Data\Excels\Only_Male\GeneCpG\SymbDigi
(307, 192589)


,Unnamed: 0,Sentrix,PATNO,HYS,Gender,APPRDX,Symbol_Digit,cg20815302,cg25795414,cg00959375,...,cg03814399,cg14792798,cg16910097,cg08750857,cg17933893,cg06917935,cg02794319,cg10702286,cg10353174,cg19590050
0,0,200991620021_R01C01,3001,2,1,1,1,0.907957,0.868910,0.752844,...,0.880496,0.928546,0.900567,0.877800,0.836823,0.216268,0.234350,0.681550,0.103758,0.837890
1,1,200973410167_R04C01,3007,2,1,1,2,0.918626,0.865257,0.798170,...,0.877306,0.912956,0.935591,0.942104,0.750613,0.254265,0.290337,0.760554,0.075480,0.833824
2,2,200973410167_R05C01,3010,2,1,1,1,0.905297,0.864778,0.769766,...,0.891733,0.902633,0.947783,0.942099,0.752775,0.229338,0.232169,0.775542,0.119109,0.846339


In [20]:
df1 = df.drop(['Unnamed: 0','PATNO', 'Sentrix', 'HYS', 'Gender', 'Symbol_Digit'], axis=1)
df1.head(2)

,APPRDX,cg20815302,cg25795414,cg00959375,cg07781847,cg05138383,cg27394794,cg12443990,cg19788329,cg22178513,...,cg03814399,cg14792798,cg16910097,cg08750857,cg17933893,cg06917935,cg02794319,cg10702286,cg10353174,cg19590050
0,1,0.907957,0.868910,0.752844,0.827413,0.771393,0.766283,0.791932,0.785104,0.723989,...,0.880496,0.928546,0.900567,0.877800,0.836823,0.216268,0.234350,0.681550,0.103758,0.837890
1,1,0.918626,0.865257,0.798170,0.823846,0.760193,0.801117,0.772456,0.742722,0.580721,...,0.877306,0.912956,0.935591,0.942104,0.750613,0.254265,0.290337,0.760554,0.075480,0.833824


In [21]:
df1 = df1.transpose()
df1.head(2)

,0,1,2,3,4,5,6,7,8,9,...,297,298,299,300,301,302,303,304,305,306
APPRDX,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,...,2.000000,2.00000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
cg20815302,0.907957,0.918626,0.905297,0.944309,0.932802,0.925859,0.93942,0.920432,0.929847,0.923633,...,0.928209,0.92906,0.916529,0.928151,0.919329,0.928507,0.938462,0.902386,0.930942,0.917123


In [22]:
df1.rename(columns=df1.iloc[0], inplace = True)


In [23]:
df1.index.name ='CpG'
df1.reset_index(inplace=True)
df1.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
0,APPRDX,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,...,2.000000,2.00000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
1,cg20815302,0.907957,0.918626,0.905297,0.944309,0.932802,0.925859,0.93942,0.920432,0.929847,...,0.928209,0.92906,0.916529,0.928151,0.919329,0.928507,0.938462,0.902386,0.930942,0.917123


In [24]:
df1 = df1.drop ([0], axis = 0)
df1.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,cg20815302,0.907957,0.918626,0.905297,0.944309,0.932802,0.925859,0.939420,0.920432,0.929847,...,0.928209,0.929060,0.916529,0.928151,0.919329,0.928507,0.938462,0.902386,0.930942,0.917123
2,cg25795414,0.868910,0.865257,0.864778,0.914462,0.900233,0.889566,0.901912,0.856261,0.886372,...,0.901662,0.826713,0.892833,0.851249,0.891322,0.900639,0.891959,0.867114,0.906443,0.886079


In [25]:
df2 = pd.read_csv ("SymbDigi_BorutaMale_Top50.csv")
df2.head(2)

,CpG
0,cg27171575
1,cg15419852


In [26]:
df3 = pd.merge (df1, df2, on= "CpG", how= "inner")
df3.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
0,cg02471954,0.853669,0.918805,0.927422,0.927488,0.924888,0.923705,0.89560,0.912052,0.881133,...,0.872829,0.895275,0.908137,0.916732,0.859401,0.899681,0.912347,0.912452,0.924511,0.902949
1,cg27171575,0.853809,0.895204,0.896647,0.927293,0.895606,0.917774,0.92623,0.927574,0.907450,...,0.914453,0.892855,0.900466,0.898508,0.917100,0.918530,0.919159,0.895306,0.884904,0.917070


In [27]:
df3 = df3.transpose()
df3.head(2)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
CpG,cg02471954,cg27171575,cg10693616,cg22065207,cg16444863,cg18174834,cg22508530,cg06129513,cg05817202,cg17349352,...,cg12101479,cg27438935,cg17091528,cg01575173,cg07404352,cg09790959,cg13468400,cg13240962,cg13590297,cg15419852
1.0,0.853669,0.853809,0.012194,0.844055,0.9315,0.040499,0.597781,0.843875,0.823899,0.045292,...,0.802371,0.012468,0.926865,0.92379,0.864344,0.936301,0.812987,0.77844,0.103243,0.02594


In [28]:
df3.rename(columns=df3.iloc[0], inplace = True)


In [29]:
df3.index.name ='APPRDX'
df3.reset_index(inplace=True)
df3.head(2)

,APPRDX,cg02471954,cg27171575,cg10693616,cg22065207,cg16444863,cg18174834,cg22508530,cg06129513,cg05817202,...,cg12101479,cg27438935,cg17091528,cg01575173,cg07404352,cg09790959,cg13468400,cg13240962,cg13590297,cg15419852
0,CpG,cg02471954,cg27171575,cg10693616,cg22065207,cg16444863,cg18174834,cg22508530,cg06129513,cg05817202,...,cg12101479,cg27438935,cg17091528,cg01575173,cg07404352,cg09790959,cg13468400,cg13240962,cg13590297,cg15419852
1,1.0,0.853669,0.853809,0.012194,0.844055,0.9315,0.040499,0.597781,0.843875,0.823899,...,0.802371,0.012468,0.926865,0.92379,0.864344,0.936301,0.812987,0.77844,0.103243,0.02594


In [30]:
df3 = df3.drop ([0], axis = 0)
df3.head(2)

,APPRDX,cg02471954,cg27171575,cg10693616,cg22065207,cg16444863,cg18174834,cg22508530,cg06129513,cg05817202,...,cg12101479,cg27438935,cg17091528,cg01575173,cg07404352,cg09790959,cg13468400,cg13240962,cg13590297,cg15419852
1,1.0,0.853669,0.853809,0.012194,0.844055,0.9315,0.040499,0.597781,0.843875,0.823899,...,0.802371,0.012468,0.926865,0.92379,0.864344,0.936301,0.812987,0.77844,0.103243,0.02594
2,1.0,0.918805,0.895204,0.012696,0.851627,0.937715,0.034218,0.546212,0.920245,0.849287,...,0.900273,0.011568,0.931374,0.909799,0.927214,0.921975,0.897971,0.799525,0.315527,0.037032


In [31]:
df3.reset_index(inplace = True, drop = True)
df3.head()

,APPRDX,cg02471954,cg27171575,cg10693616,cg22065207,cg16444863,cg18174834,cg22508530,cg06129513,cg05817202,...,cg12101479,cg27438935,cg17091528,cg01575173,cg07404352,cg09790959,cg13468400,cg13240962,cg13590297,cg15419852
0,1.0,0.853669,0.853809,0.012194,0.844055,0.9315,0.040499,0.597781,0.843875,0.823899,...,0.802371,0.012468,0.926865,0.92379,0.864344,0.936301,0.812987,0.77844,0.103243,0.02594
1,1.0,0.918805,0.895204,0.012696,0.851627,0.937715,0.034218,0.546212,0.920245,0.849287,...,0.900273,0.011568,0.931374,0.909799,0.927214,0.921975,0.897971,0.799525,0.315527,0.037032
2,1.0,0.927422,0.896647,0.011696,0.904624,0.931614,0.040836,0.54399,0.922113,0.877793,...,0.863307,0.008086,0.925293,0.891893,0.92932,0.903752,0.89016,0.764607,0.178676,0.03841
3,1.0,0.927488,0.927293,0.012665,0.875634,0.953024,0.042759,0.647819,0.902452,0.886403,...,0.904549,0.009485,0.944399,0.938138,0.939123,0.952805,0.877528,0.800614,0.349652,0.047948
4,1.0,0.924888,0.895606,0.008074,0.897773,0.919511,0.035405,0.569595,0.885714,0.86782,...,0.851145,0.005803,0.943601,0.936201,0.8373,0.941723,0.830541,0.818688,0.098345,0.02047


In [32]:
df3.shape

(307, 51)

In [33]:
df3 ['APPRDX'] = df3 ['APPRDX'].apply(int)

## RanFor 1000 times, CpG only 

In [34]:
df3.head(2)

,APPRDX,cg02471954,cg27171575,cg10693616,cg22065207,cg16444863,cg18174834,cg22508530,cg06129513,cg05817202,...,cg12101479,cg27438935,cg17091528,cg01575173,cg07404352,cg09790959,cg13468400,cg13240962,cg13590297,cg15419852
0,1,0.853669,0.853809,0.012194,0.844055,0.9315,0.040499,0.597781,0.843875,0.823899,...,0.802371,0.012468,0.926865,0.92379,0.864344,0.936301,0.812987,0.77844,0.103243,0.02594
1,1,0.918805,0.895204,0.012696,0.851627,0.937715,0.034218,0.546212,0.920245,0.849287,...,0.900273,0.011568,0.931374,0.909799,0.927214,0.921975,0.897971,0.799525,0.315527,0.037032


In [35]:
df3.shape

(307, 51)

In [36]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(90))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [37]:
dframe1.head(2)

,Accuracy,Std_Dev
0,0.664444,0.083118
1,0.666667,0.108866


In [38]:
dframe1.to_csv ("SymbDigi_BorutaCpG_Male_1000RF.csv")